https://opendata.mkrf.ru/opendata/7705851331-egrkn

In [1]:
import pandas as pd
import numpy as np

import re

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
file_path = 'C:/00_Data/MinKult/'
file_name = 'reestr_OKN_2021_12_17.csv'

In [3]:
# загружаю файл, скачанный с сайта Минкульта

work_old = pd.read_csv(file_path + file_name, encoding='utf-8', low_memory=False)

In [4]:
work_old.shape

(147290, 53)

In [5]:
# смотрю случайные строки

work_old.sample(3)

,Объект,Номер в реестре,название улицы,Комментарий,Идентификатор дома в ФИАС,FiasAuxSubobjId,FiasAuxObjId,Идентификатор улицы в ФИАС,Идентификатор населенного пункта в ФИАС,Идентификатор внутрегородской территории в ФИАС,...,Id - идентификатор,признак актуальности,признак публичности,год постановки ОКН на Госохрану,год проведения реставрации,"параметры, идентифицирующие ФЦП",относительный URL объекта,категория объекта,дата создания объекта,дата обновления объекта
94001,Памятник В.И. Ленину,231710819040005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,255738,True,NaN,NaN,NaN,NaN,/cdm/v2/heritages/1/255738,heritage,2017-03-22T06:20:01Z,NaN
50189,Дворец культуры завода резиново-технических из...,021510385740005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,211133,True,NaN,1976.0,NaN,NaN,/cdm/v2/heritages/1/211133,heritage,NaN,2020-08-18T06:16:19Z
99483,Городище у д. Городище,691740869920006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,261296,True,NaN,NaN,NaN,NaN,/cdm/v2/heritages/1/261296,heritage,2017-04-13T14:33:03Z,NaN


In [6]:
# удаляю пустые столбцы

for column_name in work_old.columns: # цикл по столбцам
    if list(work_old[column_name].value_counts()) == []: # проверка на содержательность
        work_old.drop(column_name, axis=1, inplace=True) # если пустой, удаляю

In [7]:
# нахожу столбцы в которых только 1 значение

for column_name in work_old.columns:
    if work_old[column_name].nunique() == 1:
        print('Название столбца: ' + column_name)
        #print('Содержание' + work[column_name].unique)        
        #work.drop(column_name, axis=1, inplace=True)       

Название столбца: признак актуальности
Название столбца: категория объекта


In [8]:
# удаляю

work_old.drop('категория объекта', axis=1, inplace=True)

In [9]:
# меняю названия

work_old.rename(columns={'дата создания': 'date'}, inplace=True)

In [10]:
work_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147290 entries, 0 to 147289
Data columns (total 31 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Объект                                      147290 non-null  object 
 1   Номер в реестре                             147290 non-null  object 
 2   Полный адрес                                92192 non-null   object 
 3   На карте                                    39820 non-null   object 
 4   Id-Регион                                   147290 non-null  int64  
 5   Регион                                      147290 non-null  object 
 6   учетный номер                               147290 non-null  object 
 7   Id-Категория историко-культурного значения  147290 non-null  int64  
 8   Категория историко-культурного значения     147290 non-null  object 
 9   Id-Вид объекта                              147290 non-null  int64  
 

In [11]:
work_old.describe()

,Id-Регион,Id-Категория историко-культурного значения,Id-Вид объекта,Id-Принадлежность к Юнеско,Id-Особо ценный объект,идентификатор ансамбля,Id - идентификатор,год постановки ОКН на Госохрану,год проведения реставрации
count,147290.000000,147290.000000,147290.000000,147290.000000,147290.000000,20016.000000,147290.000000,2353.000000,47.000000
mean,45.208100,1.567717,1.176753,1.994100,1.998031,224410.708533,235468.865429,1996.629834,2013.978723
std,23.425768,0.612570,0.417427,0.079882,0.049810,46198.039553,46539.049091,17.076665,6.169664
min,1.000000,1.000000,1.000000,0.000000,0.000000,147610.000000,147449.000000,1947.000000,1993.000000
25%,24.000000,1.000000,1.000000,2.000000,2.000000,186697.000000,197328.250000,1987.000000,2008.500000
50%,48.000000,2.000000,1.000000,2.000000,2.000000,215413.000000,235160.500000,1994.000000,2016.000000
75%,63.000000,2.000000,1.000000,2.000000,2.000000,256759.000000,272472.750000,2018.000000,2019.000000
max,92.000000,4.000000,3.000000,2.000000,2.000000,328492.000000,329115.000000,2021.000000,2020.000000


## Даты ##

In [12]:
# удаляю двойные пробелы

work_old['date'] = work_old['date'].str.replace('\s+', ' ', regex=True )

In [13]:
# заполняю пропуски, чтобы избежать ошибок

work_old['date'].fillna('none', inplace=True)

In [14]:
# удаляю пробелы в конце и начале строк

work_old['date'] = work_old['date'].apply(lambda x: x.strip())

In [15]:
work = work_old.copy()

In [16]:
# меняю "в. н. э." и "век н.э." на просто "век"

work['date'] = work['date'].str.replace(r'\s?в?в\.?\s?н\.?\s?э\.?', ' век', regex=True)
work['date'] = work['date'].str.replace(r'век\s?н\.?\s?э\.?', ' век', regex=True)
work['date'] = work['date'].str.replace(r'века\s?нашей эры', ' век', regex=True)
work['date'] = work['date'].str.replace(r'вв.\s?нашей эры', ' век', regex=True)
work['date'] = work['date'].str.replace(r'тысячелети[я|е] нашей эры', 'тыс.', regex=True)

In [23]:
work['Полный адрес'].fillna('none', inplace=True)

In [29]:
work[work['Полный адрес'].str.contains('Вязьма', case=True)].to_excel(file_path + 'temp.xlsx')

In [ ]:
(work[work['date'].str.contains(r'\sэры', regex=True)]
 .date.value_counts().reset_index()
)#.to_excel(file_path + 'temp.xlsx')

In [ ]:
# удаляю точки в конце строки (в некоторых строках 2 точки в конце)

work['date'] = work['date'].str.replace(r'\.{1,3}$', '', regex=True)

In [ ]:
work['date'] = work['date'].str.replace(r'\sв$', ' век', regex=True)

In [ ]:
work['date'] = work['date'].str.replace(r'\sв\sн.э$', ' век', regex=True)

In [ ]:
work['date'].value_counts()

### Чищу данные ###

Меняю обозначение века на "век"

In [ ]:
(work[work['date'].str.contains(r'^\w{2,5}\s{0,1}в\.{0,1}(\sн\.э\.){0,1}$', regex=True, na=False)]
 .date.value_counts().reset_index()
)
#.to_excel(file_path + 'temp.xlsx')

In [ ]:
(
    work[work['date'].str.contains(r'^\w{2,5}\sвек$', regex=True, na=False)]
    .date.nunique()
)

In [ ]:
# смотрю что есть

(work[work['date'].str.contains(r'^\w{2,5}\s{0,1}в.{0,1}[\sн.\sэ]{0,4}$', regex=True, na=False)]
 .date.value_counts().reset_index()
)

#.to_excel(file_path + 'temp.xlsx')

In [ ]:
# количество

work[work['date'].str.contains(r'^\w{2,5}\s{0,1}в.{0,1}[\sн.\sэ]{0,5}$', regex=True, na=False)].date.nunique()

In [ ]:
# меняю

work['date'] = work['date'].str.replace(r'в.{0,1}[\sн.\sэ]{0,5}$', 'век', regex=True)

In [ ]:
work[work['date'].str.contains(r'^\w{2,5}\s{0,1}век$', regex=True, na=False)].date.value_counts().sum()

In [ ]:
work[work['date'].str.contains(r'^\w{3}\sв.$', regex=True, na=False)].shape

In [ ]:
(work[work['date'].str.contains(r'^\w{2,5}\sв[\D]{0,6}$', regex=True, na=False)]
 .date.value_counts().reset_index())['index'].unique()

In [ ]:
work.to_excel('C:/00_Data/OKN_50.xlsx', encoding='utf-8', index=False)

In [ ]:
# сколько обектов имеют точную дату создания: четыре цифры и приписку "г."

work[work['date'].str.contains(r'^\d{4}\sг.$', regex= True, na=False)].shape

In [ ]:
# у некоторых строк на конце нет точки 

work[work['date'].str.contains(r'^\d{4}\sг$', regex= True, na=False)].shape

In [ ]:
# а у некоторых нет и буквы "г"

work[work['date'].str.contains(r'^\d{4}$', case=True, regex= True, na=False)].shape

In [ ]:
# удаляю "г."" и "г" в конце строки, чтобы отфильтровать ОКН с точной датой создания

work['date'] = work['date'].str.replace(r'\sг$','', regex= True)
work['date'] = work['date'].str.replace(r'\sг.$','', regex= True)

In [ ]:
# доля объектов с точной датой

int((work[work['date'].str.contains(r'^\d{4}$', case=True, regex= True, na=False)].shape[0] /
 work.shape[0]) * 100)

In [ ]:
work['first_year'] = work[work['date'].str.contains(r'^\d{4}$', case=True, regex= True, na=False)].date

In [ ]:
wor

In [ ]:
work[work['дата создания'].str.contains(r'^\d{4}$', case=True, regex= True, na=False)].shape

In [ ]:
temp = work[work['дата создания'].str.contains(r'^\d{4}$', case=True, regex= True, na=False)]

In [ ]:
temp = work[work['дата создания'].str.contains(r'\sг$', regex= True, na=False)]

In [ ]:
temp['дата создания'].value_counts

In [ ]:
work['дата создания'].value_counts().reset_index().head(25)

In [ ]:
# смотрю столбцы
work.columns.to_list()

In [ ]:
# список с ненужными столбцами

unless_feature = [
'Id',
'учетный номер',
'Id.1',
'Id.3',
'текстовое описание границ',
'Id.5',
'Id.6',
'идентификатор ансамбля',
'зоны охраны и режимы использования земель',
'Items',
'Id.7',
'признак актуальности',
'категория объекта',
#'дата создания объекта',
#'дата обновления объекта'    
]

In [ ]:
# удаляю ненужные столбцы

for column_name in unless_feature:
    work.drop(column_name, axis=1, inplace=True)

In [ ]:
# список ненужных дат

unless_dates = [
    'XI',
    'XV',
    'XX в.'    
]

In [ ]:
# удаляю ненужные столбцы

for column_name in unless_dates:
    work.drop(column_name, axis=1, inplace=True)

In [ ]:
work.shape

In [ ]:
# удаляю строки с лишними датами

for unless in unless_dates:
    work = work[~(work['дата создания'].str.contains(unless, case=False))]

In [ ]:
work.dtypes

In [ ]:
work = pd.read_excel('C:/00_Projects/2021/OKN/OKN_2021_03_10.xlsx')

In [ ]:
work.sample()

In [ ]:
work.shape

In [ ]:
for date in range(1099, 1979):
     work = work[~(work.date == date)]   

In [ ]:
work.shape

In [ ]:
work.to_excel('C:/00_Data/OKN_50.xlsx', encoding='utf-8', index=False)

In [ ]:
temp.to_excel('C:/00_Data/temp.xlsx', encoding='utf-8', index=False)